In [9]:
import pandas as pd 
import requests
import json
from datetime import datetime

In [10]:
JSON_URL = "https://wowtokenprices.com/history_prices_full.json"

In [11]:
def request_token_price(request):
    response = requests.get(request)
    response_json = response.json()
    return response_json['eu']

In [12]:
df = pd.DataFrame(request_token_price(JSON_URL))
df

,price,time
0,35000,1429646660
1,35350,1429647590
2,35704,1429648533
3,36061,1429649396
4,36422,1429650302
...,...,...
133153,166084,1610355885
133154,166089,1610357085
133155,165951,1610358285
133156,165854,1610359485


In [13]:
df['date']=df['time'].apply(lambda x: datetime.fromtimestamp(x))
df['day']=df['date'].apply(lambda x: x.weekday())
df['hour']=df['date'].apply(lambda x: x.hour)
df['date']=pd.to_datetime(df['date'])

In [14]:
df=df.set_index('date',drop=False)
df = df.sort_index()
df

,price,time,date,day,hour
date,,,,,
2015-04-21 22:04:20,35000,1429646660,2015-04-21 22:04:20,1,22
2015-04-21 22:19:50,35350,1429647590,2015-04-21 22:19:50,1,22
2015-04-21 22:35:33,35704,1429648533,2015-04-21 22:35:33,1,22
2015-04-21 22:49:56,36061,1429649396,2015-04-21 22:49:56,1,22
2015-04-21 23:05:02,36422,1429650302,2015-04-21 23:05:02,1,23
...,...,...,...,...,...
2021-01-11 10:04:45,166084,1610355885,2021-01-11 10:04:45,0,10
2021-01-11 10:24:45,166089,1610357085,2021-01-11 10:24:45,0,10
2021-01-11 10:44:45,165951,1610358285,2021-01-11 10:44:45,0,10


In [15]:
std = lambda x: (x.std())
df['7_day_std']=df['price'].rolling('7d').apply(std)
df['30_day_std']=df['price'].rolling('30d').apply(std)
df['7_day_mean']=df['price'].rolling('7d').mean()
df['30_day_mean']=df['price'].rolling('30d').mean()
df['7_day_low']=df['price'].rolling('7d').min()
df['30_day_low']=df['price'].rolling('30d').min()
df['7_day_high']=df['price'].rolling('7d').max()
df['30_day_high']=df['price'].rolling('30d').max()
df['1_day_high']=df['price'].rolling('1d').max()
df['1_day_low']=df['price'].rolling('1d').min()
df['last_change']=df['price'].diff(periods=2)
df=df.dropna()

In [16]:
df=df.rename(columns = {'price':'current_price'}, inplace = False)
df

,current_price,time,date,day,hour,7_day_std,30_day_std,7_day_mean,30_day_mean,7_day_low,30_day_low,7_day_high,30_day_high,1_day_high,1_day_low,last_change
date,,,,,,,,,,,,,,,,
2015-04-21 22:35:33,35704,1429648533,2015-04-21 22:35:33,1,22,352.001894,352.001894,35351.333333,35351.333333,35000.0,35000.0,35704.0,35704.0,35704.0,35000.0,704.0
2015-04-21 22:49:56,36061,1429649396,2015-04-21 22:49:56,1,22,456.629226,456.629226,35528.750000,35528.750000,35000.0,35000.0,36061.0,36061.0,36061.0,35000.0,711.0
2015-04-21 23:05:02,36422,1429650302,2015-04-21 23:05:02,1,23,562.104795,562.104795,35707.400000,35707.400000,35000.0,35000.0,36422.0,36422.0,36422.0,35000.0,718.0
2015-04-21 23:20:38,36786,1429651238,2015-04-21 23:20:38,1,23,668.330582,668.330582,35887.166667,35887.166667,35000.0,35000.0,36786.0,36786.0,36786.0,35000.0,725.0
2015-04-21 23:35:11,37154,1429652111,2015-04-21 23:35:11,1,23,775.556666,775.556666,36068.142857,36068.142857,35000.0,35000.0,37154.0,37154.0,37154.0,35000.0,732.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-11 10:04:45,166084,1610355885,2021-01-11 10:04:45,0,10,2650.981297,2654.819893,159799.816733,160392.395307,155921.0,155921.0,167871.0,170309.0,166087.0,156615.0,166.0
2021-01-11 10:24:45,166089,1610357085,2021-01-11 10:24:45,0,10,2641.327511,2658.844467,159796.266932,160394.863718,155921.0,155921.0,167839.0,170309.0,166089.0,156615.0,2.0
2021-01-11 10:44:45,165951,1610358285,2021-01-11 10:44:45,0,10,2631.177345,2662.567180,159792.505976,160397.180505,155921.0,155921.0,167646.0,170309.0,166089.0,156615.0,-133.0


In [17]:
df.to_csv("Token-prices.csv",index=False)

In [18]:
df['last_change'].max()

46481.0